In [1]:
import pyspark
from pyspark.sql.types import *
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import os
from pyspark.ml.feature import StringIndexer, OneHotEncoder
from pyspark.sql.types import StringType, StructType, StructField


# Cria uma sessão Spark habilitando Hive support para armazenar dados no Spoark Warehouse
spark = SparkSession \
    .builder \
    .appName("projeto_parte_ii") \
    .config('spark.master', 'local') \
    .enableHiveSupport() \
    .getOrCreate()

24/06/09 16:34:54 WARN Utils: Your hostname, hadoop-lubuntu resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
24/06/09 16:34:54 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/09 16:34:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
# Spark read Hive table
spark.sql("USE projeto")

24/06/09 16:35:39 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
24/06/09 16:35:39 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
24/06/09 16:35:50 WARN ObjectStore: Version information not found in metastore. hive.metastore.schema.verification is not enabled so recording the schema version 2.3.0
24/06/09 16:35:50 WARN ObjectStore: setMetaStoreSchemaVersion called but recording version is disabled: version = 2.3.0, comment = Set by MetaStore hadoop@127.0.1.1
24/06/09 16:35:51 WARN ObjectStore: Failed to get database global_temp, returning NoSuchObjectException


DataFrame[]

In [3]:
df = spark.sql("SELECT * FROM notas_fiscais LIMIT 100000")
df.printSchema()

root
 |-- nf_numero: string (nullable = true)
 |-- nf_data_emissao: date (nullable = true)
 |-- nf_valor_total: decimal(15,2) (nullable = true)
 |-- emit_cnpj: string (nullable = true)
 |-- emit_cep: string (nullable = true)
 |-- emit_municipio: string (nullable = true)
 |-- dest_cnpj: string (nullable = true)
 |-- dest_cep: string (nullable = true)
 |-- dest_municipio: string (nullable = true)
 |-- prod_nr_item: integer (nullable = true)
 |-- prod_cod: string (nullable = true)
 |-- prod_desc: string (nullable = true)
 |-- prod_ncm: string (nullable = true)
 |-- ano: integer (nullable = true)
 |-- nome_regiao: string (nullable = true)
 |-- sigla_uf: string (nullable = true)
 |-- nome_municipio: string (nullable = true)
 |-- nome_mesoregiao: string (nullable = true)
 |-- nome_microregiao: string (nullable = true)
 |-- tipologia_rural_urbana: string (nullable = true)
 |-- hierarquia_urbana: string (nullable = true)
 |-- pop: integer (nullable = true)
 |-- pib: decimal(15,2) (nullable = t

In [4]:
columns_to_keep = '''
prod_ncm, 
prod_unid_nn as prod_unid, 
scaled_log_prod_quant as prod_quant, 
scaled_log_prod_valor_unit as prod_valor_unit,
nome_mesoregiao as reg_mesoregiao, 
tipologia_rural_urbana as reg_tipologia, 
hierarquia_urbana as reg_hierarquia, 
scaled_pop_meso as reg_pop,
scaled_pib_meso as reg_pib
'''

df = spark.sql(f"SELECT {columns_to_keep} FROM notas_fiscais")
df.printSchema()

root
 |-- prod_ncm: string (nullable = true)
 |-- prod_unid: string (nullable = true)
 |-- prod_quant: double (nullable = true)
 |-- prod_valor_unit: double (nullable = true)
 |-- reg_mesoregiao: string (nullable = true)
 |-- reg_tipologia: string (nullable = true)
 |-- reg_hierarquia: string (nullable = true)
 |-- reg_pop: double (nullable = true)
 |-- reg_pib: double (nullable = true)



In [5]:
#df.show()

In [6]:
# prod_ncm
indexer = StringIndexer(inputCol="prod_ncm", outputCol="prod_ncm_indexed")
df = indexer.fit(df).transform(df)

encoder = OneHotEncoder(inputCol="prod_ncm_indexed", outputCol="prod_ncm_onehot")
df = encoder.fit(df).transform(df)

#prod_unid
indexer = StringIndexer(inputCol="prod_unid", outputCol="prod_unid_indexed")
df = indexer.fit(df).transform(df)

encoder = OneHotEncoder(inputCol="prod_unid_indexed", outputCol="prod_unid_onehot")
df = encoder.fit(df).transform(df)

#reg_mesoregiao
indexer = StringIndexer(inputCol="reg_mesoregiao", outputCol="reg_mesoregiao_indexed")
df = indexer.fit(df).transform(df)

encoder = OneHotEncoder(inputCol="reg_mesoregiao_indexed", outputCol="reg_mesoregiao_onehot")
df = encoder.fit(df).transform(df)

#reg_tipologia
indexer = StringIndexer(inputCol="reg_tipologia", outputCol="reg_tipologia_indexed")
df = indexer.fit(df).transform(df)

encoder = OneHotEncoder(inputCol="reg_tipologia_indexed", outputCol="reg_tipologia_onehot")
df = encoder.fit(df).transform(df)

#reg_hierarquia
indexer = StringIndexer(inputCol="reg_hierarquia", outputCol="reg_hierarquia_indexed")
df = indexer.fit(df).transform(df)

encoder = OneHotEncoder(inputCol="reg_hierarquia_indexed", outputCol="reg_hierarquia_onehot")
df = encoder.fit(df).transform(df)

#df.show(truncate=False)

In [7]:
df.printSchema()

root
 |-- prod_ncm: string (nullable = true)
 |-- prod_unid: string (nullable = true)
 |-- prod_quant: double (nullable = true)
 |-- prod_valor_unit: double (nullable = true)
 |-- reg_mesoregiao: string (nullable = true)
 |-- reg_tipologia: string (nullable = true)
 |-- reg_hierarquia: string (nullable = true)
 |-- reg_pop: double (nullable = true)
 |-- reg_pib: double (nullable = true)
 |-- prod_ncm_indexed: double (nullable = false)
 |-- prod_ncm_onehot: vector (nullable = true)
 |-- prod_unid_indexed: double (nullable = false)
 |-- prod_unid_onehot: vector (nullable = true)
 |-- reg_mesoregiao_indexed: double (nullable = false)
 |-- reg_mesoregiao_onehot: vector (nullable = true)
 |-- reg_tipologia_indexed: double (nullable = false)
 |-- reg_tipologia_onehot: vector (nullable = true)
 |-- reg_hierarquia_indexed: double (nullable = false)
 |-- reg_hierarquia_onehot: vector (nullable = true)



In [8]:
df.createOrReplaceTempView("df")

columns_to_keep = '''
prod_ncm_onehot as prod_ncm,
prod_unid_onehot as prod_unid,
prod_quant,
prod_valor_unit,
reg_mesoregiao_onehot as reg_mesoregiao,
reg_tipologia_onehot as reg_tipologia,
reg_hierarquia_onehot as reg_hierarquia,
reg_pop,
reg_pib
'''

df = spark.sql(f"SELECT {columns_to_keep} FROM df")
df.printSchema()

root
 |-- prod_ncm: vector (nullable = true)
 |-- prod_unid: vector (nullable = true)
 |-- prod_quant: double (nullable = true)
 |-- prod_valor_unit: double (nullable = true)
 |-- reg_mesoregiao: vector (nullable = true)
 |-- reg_tipologia: vector (nullable = true)
 |-- reg_hierarquia: vector (nullable = true)
 |-- reg_pop: double (nullable = true)
 |-- reg_pib: double (nullable = true)



24/06/09 16:36:24 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [9]:
#df.show()

In [10]:
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import RegressionEvaluator

In [11]:
assembler = VectorAssembler(
    inputCols=["prod_ncm", "prod_unid", "prod_quant", "reg_mesoregiao", "reg_tipologia", "reg_hierarquia", "reg_pop", "reg_pib"],
    outputCol="features")

df_lr = assembler.transform(df)
final_df = df_lr.select("features", "prod_valor_unit")

train_df, test_df = final_df.randomSplit([0.8, 0.2], seed=26)

In [12]:
linear_regressor = LinearRegression(featuresCol="features", labelCol="prod_valor_unit", predictionCol="predicted_prod_valor_unit_lr")
linear_regressor_model = linear_regressor.fit(train_df)

24/06/09 16:36:28 WARN DAGScheduler: Broadcasting large task binary with size 1619.6 KiB
24/06/09 16:37:14 WARN DAGScheduler: Broadcasting large task binary with size 1620.2 KiB
24/06/09 16:37:40 WARN DAGScheduler: Broadcasting large task binary with size 1620.2 KiB
24/06/09 16:37:40 WARN DAGScheduler: Broadcasting large task binary with size 1620.2 KiB
24/06/09 16:37:41 WARN DAGScheduler: Broadcasting large task binary with size 1620.2 KiB
24/06/09 16:37:41 WARN DAGScheduler: Broadcasting large task binary with size 1620.2 KiB
24/06/09 16:37:41 WARN DAGScheduler: Broadcasting large task binary with size 1620.2 KiB
24/06/09 16:37:42 WARN DAGScheduler: Broadcasting large task binary with size 1620.2 KiB
24/06/09 16:37:42 WARN DAGScheduler: Broadcasting large task binary with size 1620.2 KiB
24/06/09 16:37:42 WARN DAGScheduler: Broadcasting large task binary with size 1620.2 KiB
24/06/09 16:37:42 WARN DAGScheduler: Broadcasting large task binary with size 1620.2 KiB
24/06/09 16:37:43 WAR

In [13]:
predictions_lr = linear_regressor_model.transform(test_df)

evaluator = RegressionEvaluator(labelCol="prod_valor_unit", predictionCol="predicted_prod_valor_unit_lr", metricName="rmse")
rmse = evaluator.evaluate(predictions_lr)
print("Root Mean Squared Error (RMSE) on test data: {:.3f}".format(rmse))

evaluator_r2 = RegressionEvaluator(labelCol="prod_valor_unit", predictionCol="predicted_prod_valor_unit_lr", metricName="r2")
r2 = evaluator_r2.evaluate(predictions_lr)
print("R-squared (R2) on test data: {:.3f}".format(r2))

24/06/09 16:38:18 WARN DAGScheduler: Broadcasting large task binary with size 1676.2 KiB


Root Mean Squared Error (RMSE) on test data: 0.560


24/06/09 16:38:50 WARN DAGScheduler: Broadcasting large task binary with size 1676.2 KiB


R-squared (R2) on test data: 0.687


In [14]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.regression import DecisionTreeRegressor
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

In [15]:
assembler = VectorAssembler(
    inputCols=["prod_ncm", "prod_unid", "prod_quant", "reg_mesoregiao", "reg_tipologia", "reg_hierarquia", "reg_pop", "reg_pib"],
    outputCol="features")

df_dt = assembler.transform(df)
final_df = df_dt.select("features", "prod_valor_unit")

train_df, test_df = final_df.randomSplit([0.8, 0.2], seed=26)

In [16]:
tree_regressor = DecisionTreeRegressor(featuresCol="features", labelCol="prod_valor_unit", predictionCol="predicted_prod_valor_unit_tr", maxDepth=6)
tree_regressor_model = tree_regressor.fit(train_df)

24/06/09 16:39:21 WARN DAGScheduler: Broadcasting large task binary with size 1618.3 KiB
24/06/09 16:39:46 WARN DAGScheduler: Broadcasting large task binary with size 1618.4 KiB
24/06/09 16:40:04 WARN DAGScheduler: Broadcasting large task binary with size 1623.2 KiB
24/06/09 16:40:22 WARN DAGScheduler: Broadcasting large task binary with size 1861.8 KiB
24/06/09 16:40:38 WARN MemoryStore: Not enough space to cache rdd_158_0 in memory! (computed 170.1 MiB so far)
24/06/09 16:40:38 WARN BlockManager: Persisting block rdd_158_0 to disk instead.
24/06/09 16:44:38 WARN MemoryStore: Not enough space to cache rdd_158_0 in memory! (computed 255.3 MiB so far)
24/06/09 16:46:52 WARN DAGScheduler: Broadcasting large task binary with size 1862.6 KiB
24/06/09 16:46:52 WARN MemoryStore: Not enough space to cache rdd_158_0 in memory! (computed 255.3 MiB so far)
24/06/09 16:49:15 WARN DAGScheduler: Broadcasting large task binary with size 1863.3 KiB
24/06/09 16:49:15 WARN MemoryStore: Not enough space

In [17]:
predictions_tr = tree_regressor_model.transform(test_df)

evaluator = RegressionEvaluator(labelCol="prod_valor_unit", predictionCol="predicted_prod_valor_unit_tr", metricName="rmse")
rmse = evaluator.evaluate(predictions_tr)
print("Root Mean Squared Error (RMSE) on test data: {:.3f}".format(rmse))

evaluator_r2 = RegressionEvaluator(labelCol="prod_valor_unit", predictionCol="predicted_prod_valor_unit_tr", metricName="r2")
r2 = evaluator_r2.evaluate(predictions_tr)
print("R-squared (R2) on test data: {:.3f}".format(r2))

24/06/09 16:59:11 WARN DAGScheduler: Broadcasting large task binary with size 1621.4 KiB


Root Mean Squared Error (RMSE) on test data: 0.756


24/06/09 16:59:34 WARN DAGScheduler: Broadcasting large task binary with size 1621.4 KiB


R-squared (R2) on test data: 0.430


In [18]:
feature_importance = tree_regressor_model.featureImportances.toArray()

# Show feature importance
for i, column in enumerate(assembler.getInputCols()):
    print(f"Feature '{column}': {feature_importance[i]}")

Feature 'prod_ncm': 0.0
Feature 'prod_unid': 0.0
Feature 'prod_quant': 2.4376594880005456e-05
Feature 'reg_mesoregiao': 0.0
Feature 'reg_tipologia': 5.749129765135653e-05
Feature 'reg_hierarquia': 0.0032917995968781244
Feature 'reg_pop': 0.0
Feature 'reg_pib': 0.0


In [19]:
coefficients = linear_regressor_model.coefficients
intercept = linear_regressor_model.intercept

feature_importance_lr = sorted(list(zip(df_lr.columns[:-1], map(abs, coefficients))), key=lambda x: x[1], reverse=True)

print("Feature Importance:")
for feature, importance in feature_importance_lr:
    print("  {}: {:.3f}".format(feature, importance))

Feature Importance:
  prod_valor_unit: 0.837
  reg_tipologia: 0.728
  reg_hierarquia: 0.406
  reg_pop: 0.395
  reg_pib: 0.344
  prod_unid: 0.260
  prod_ncm: 0.235
  prod_quant: 0.112
  reg_mesoregiao: 0.029
